In [1]:
!pip install torch==1.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchvision==0.8.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html                       

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.3MB 19kB/s 
ERROR: torchvision 0.9.0+cu101 has requirement torch==1.8.0, but you'll have torch 1.7.0+cu101 which is incompatible.
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.7.0+cu101 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 12.7MB 149kB/s 
  Found existing installation: torchvision 0.9.0+cu101
    Uninstalling torchvision-0.9.0+cu101:
      Successfully uninstalled torchvision-0.9.0+cu101


In [2]:
# installing dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
!pip install flask_ngrok
print(torch.__version__, torch.cuda.is_available())
print(torchvision.__version__)
!gcc --version
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime
# opencv is pre-installed on colab

     |████████████████████████████████| 276kB 10.0MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=47ae0c647d4292d2d25ea882fb243c693fd00b5758b01291967c1e37e46c96e7
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.7.0+cu101 True
0.8.1+cu101
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
     |████████████████████████████████| 6.8MB 58.0MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 2.2MB 17.5MB/s 
  Crea

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/casting_deploy_flask/new_experiment

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1a8a_R3QQUfs3GWY4Om-NuNjPy3eeJ8PP/new_experiment


In [ ]:
##########################importing important packages
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from PIL import Image
import io
import random
from flask_ngrok import run_with_ngrok
import os
# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from flask import Flask, request, render_template, jsonify
###############################3 regestring dataset for labels of the output
from detectron2.data.datasets import register_coco_instances
import pickle
from google.colab.patches import cv2_imshow
register_coco_instances("my_dataset_train10", {}, "/content/drive/MyDrive/casting_deploy_flask/new_experiment/train/_annotations.coco.json", "/content/drive/MyDrive/casting_deploy_flask/new_experiment/train/")

################################importing model
#------------------------------- importing saved model 

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train10",)
# cfg.DATASETS.VAL=("cast_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
# cfg.MODEL.ROI_BOX_CASCADE_HEAD.IOUS=(0.7,0.8,0.9)
# for i in range(10):             #beacuse checkpoint willbe made after 1000 iterations
cfg.SOLVER.MAX_ITER = 20000   #  iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# cfg.MODEL.RETINANET.NUM_CLASSES = 3
# cfg.MODEL.BACKBONE.FREEZE_AT=5
cfg.TEST.EVAL_PERIOD = 2000 
#cfg.CHECKPOINT_PERIOD =3
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
'''trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=True)'''

cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/casting_deploy_flask/new_experiment/output/model_final.pth"  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)



'''FLASK APP CODE'''

app = Flask(__name__)
run_with_ngrok(app)  

@app.route('/hello')
def hello():
	#-------------------------------making perdition
	im = cv2.imread('do2.jpg')
	outputs = predictor(im)
	v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg2.DATASETS.TRAIN[0]), scale=1)
	out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
	#cv2_imshow(out.get_image()[:, :, ::-1])
	cv2.imwrite("testit.jpg",out.get_image()[:, :, ::-1])
	return 'Hello World!'
	
	
@app.route('/predict', methods=['POST','GET'])
def predict():
  file = request.files['file']
  img_bytes = file.read()
  img_bytes = Image.open(io.BytesIO(img_bytes)).convert('RGB') 
  img_bytes.save("geeks.jpg") 
  img_bytes = np.array(img_bytes) 
  img_bytes = img_bytes[:, :, ::-1].copy() 
  outputs = predictor(img_bytes)


  v = Visualizer(img_bytes[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1)
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  #cv2_imshow(out.get_image()[:, :, ::-1])
  count=random.randint(0,1000)
  cv2.imwrite("/content/drive/MyDrive/casting_deploy_flask/new_experiment/static/testit"+str(count)+".jpg",out.get_image()[:, :, ::-1])
  return render_template("display.html",result = "/static/testit"+str(count)+".jpg")
	
@app.route('/')
def main():
  #os.remove("/content/drive/MyDrive/casting_deploy_flask/new_experiment/static/testit.jpg")
  return render_template("index.html")
@app.route('/test')
def test():
	return jsonify({'image_link': 'https://c1eb31bb279f.ngrok.io/static/testit.jpg'})
    

if __name__ == '__main__':
   app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b2cb2f3bd1f9.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [09/Mar/2021 13:34:08] "GET /robots.txt HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2021 13:34:09] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2021 13:34:09] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.7/dist-packages/detectron2/modeling/roi_heads/fast_rcnn.py:124: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  filter_inds = filter_mask.nonzero()
INFO:werkzeug:127.0.0.1 - - [09/Mar/2021 13:40:37] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2021 13:40:38] "GET /static/testit777.jpg HTTP/1.1" 200 -
